In [1]:
%pip install  edge-ml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
from edgeml import DatasetReceiver

In [4]:
%%markdown
project=DatasetReceiver("https://beta.edge-ml.org","02a4fad735d3308b68672ddb7593f047")
project.loadData()

project=DatasetReceiver("https://beta.edge-ml.org","02a4fad735d3308b68672ddb7593f047")
project.loadData()


In [5]:
%%markdown
import pickle
pickle.dump(project, open("data_snapshot/project_css25.pkl","wb"))

import pickle
pickle.dump(project, open("data_snapshot/project_css25.pkl","wb"))


In [6]:
import pickle
project=pickle.load(open("data_snapshot/project_css25.pkl","rb"))

ModuleNotFoundError: No module named 'numpy._core'

In [ ]:
df=pd.concat([d.data.iloc[40:-40].dropna().assign(**d.metaData) for d  in project.datasets if (d.name == "devicemotion" and d.data.shape[0]>0) and d.timeSeries[0].end-d.timeSeries[0].start>10000] )
df

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df["activity"])

In [ ]:
df2=df[df.activity!="testing"]
df2=pd.concat([df2.drop(columns=["mobile","browser"]),pd.get_dummies(df2[["browser"]])], axis=1)

df2.index=df2.time
df2=df2.drop(columns="time")
df2["activity"],labels=df2["activity"].factorize()
df2

In [ ]:
df2.dtypes

In [ ]:
def mode(x): 
    return (x.mode()[0])

aggs={ 
      **dict.fromkeys(df2.select_dtypes(exclude=[np.float32]).columns, [ mode ]),
      **dict.fromkeys(df2.select_dtypes(include=[np.float32]).columns, ["mean","var","min","max","median"]),
      "activity": [mode,'count'],
}

del(aggs["participantId"])
aggs

In [ ]:
df3=df2.groupby("participantId").rolling("1s").agg(aggs)
df3.columns = [col[0] + "_" + col[1] if col [1] != "mode" else col[0] for col in df3.columns]
df3

In [ ]:
df4=df3[df3.activity_count >= 60 ].copy()
df4.index=df4.index.droplevel(1)
df4=df4.reset_index()

df4.dropna(inplace=True)

df4.activity=df4.activity.apply(lambda x: labels[int(x)])
df4

In [ ]:
pickle.dump(df4,open("../features.pkl","wb"))

In [ ]:
%pip install imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X, y = RandomUnderSampler().fit_resample(df4.drop(columns=["activity"]), df4.activity)
groups=X.participantId
X.drop(columns=["participantId"], inplace=True)

In [ ]:
import seaborn as sns
df5=X[[col for col in X.columns if col.startswith("acceleration.x_")]]
df5["activity"]=y

sns.pairplot(df5, hue="activity")

In [ ]:
import warnings
from sklearn.model_selection import cross_validate,LeaveOneGroupOut
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings('ignore')
classifier_metrics = [
    "accuracy", "balanced_accuracy", "precision_macro", "recall_macro",
    "f1_macro"
]

scores=pd.DataFrame(cross_validate(DecisionTreeClassifier(), X, y, cv=LeaveOneGroupOut(), groups=groups, scoring=classifier_metrics , verbose = True))

scores

In [ ]:
scores.mean()

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GroupKFold

rfecv=RFECV(estimator=DecisionTreeClassifier(), cv=GroupKFold(n_splits=5), step=1 , verbose=1, scoring='f1_macro').fit(X,y,groups=groups)


In [ ]:
# Print the optimal number of features and their indices
print("Optimal number of features : %d" % rfecv.n_features_)
print("Selected features : %s" % rfecv.support_)
print("Feature ranking : %s" % rfecv.ranking_)

In [ ]:
cv_results = pd.DataFrame(rfecv.cv_results_)
cv_results


In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test accuracy")
plt.errorbar(
    x=cv_results.index,
    y=cv_results["mean_test_score"],
    yerr=cv_results["std_test_score"],
)
plt.title("Recursive Feature Elimination")
plt.show()

In [ ]:
df6=pd.concat([df4.participantId,df4.activity,df4[rfecv.get_feature_names_out(X.columns)]],axis=1).dropna()
df6